## Importing the Data & Libraries

In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
import os
from bs4 import BeautifulSoup
%matplotlib inline

df = pd.read_csv('train.csv')
df = df[pd.notnull(df['variety'])]

val_set = pd.read_csv("test.csv")
val_set = val_set[pd.notnull(val_set['review_description'])]
print(df.head(10))
print(df['review_description'].apply(lambda x: len(x.split(' '))).sum())

       user_name    country  \
0            NaN  Australia   
1  @wawinereport         US   
2            NaN      Italy   
3     @vossroger     France   
4     @vossroger     France   
5    @wineschach  Argentina   
6  @wawinereport         US   
7        @vboone         US   
8    @paulgwine          US   
9   @kerinokeefe      Italy   

                                        review_title  \
0  Andrew Peace 2007 Peace Family Vineyard Chardo...   
1  North by Northwest 2014 Red (Columbia Valley (...   
2                  Renato Ratti 2007 Conca  (Barolo)   
3  Domaine l'Ancienne Cure 2010 L'Abbaye White (B...   
4  Château du Cèdre 2012 Le Cèdre Vintage Malbec ...   
5  Andean 2005 Finca La Escondida Reserva Caberne...   
6  Gordon Estate 2012 Tradition Red (Columbia Val...   
7  Rock Wall 2014 Monte Rosso Reserve Zinfandel (...   
8  Elizabeth Chambers 2014 Falcon Glen Vineyard P...   
9  Mannucci Droandi 2009 Ceppeto Riserva  (Chiant...   

                                  review_

In [2]:
def print_plot(index):
    example = df[df.index == index][['review_description', 'variety']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])
print_plot(10)

With an alluring bouquet of fresh-cut jasmine and citrus, this is a light, vibrant wine, bristling in acidity that's fruity and briny in equal measure. Clean, crisp and fermented in stainless steel, it offers everything one wants in a refreshing quaff.
Tag: Sauvignon Blanc


## Text Pre-processing

In [3]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['review_description'] = df['review_description'].apply(clean_text)
val_set['review_description']=val_set['review_description'].apply(clean_text)
print_plot(10)

alluring bouquet freshcut jasmine citrus light vibrant wine bristling acidity thats fruity briny equal measure clean crisp fermented stainless steel offers everything one wants refreshing quaff
Tag: Sauvignon Blanc


In [4]:
df['review_description'].apply(lambda x: len(x.split(' '))).sum()

2067770

Partitioning the Dataset into Training & Test

In [5]:
X = df.review_description
y = df.variety
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

## Naive Bayes Classifier for Multinomial Models

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=None))

accuracy 0.43999516090007257


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                            precision    recall  f1-score   support

  Bordeaux-style Red Blend       0.56      0.56      0.56      1719
Bordeaux-style White Blend       0.00      0.00      0.00       249
            Cabernet Franc       0.00      0.00      0.00       320
        Cabernet Sauvignon       0.43      0.64      0.52      2251
           Champagne Blend       0.00      0.00      0.00       335
                Chardonnay       0.39      0.96      0.56      2813
                     Gamay       0.00      0.00      0.00       246
            Gewürztraminer       0.00      0.00      0.00       255
          Grüner Veltliner       0.00      0.00      0.00       309
                    Malbec       1.00      0.00      0.00       657
                    Merlot       0.00      0.00      0.00       748
                  Nebbiolo       0.92      0.12      0.22       696
              Pinot Grigio       0.00      0.00      0.00       251
                Pinot Gris       0.00      0.00

## Linear Support Vector Machine


In [7]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=None))

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


accuracy 0.6308170013710783
                            precision    recall  f1-score   support

  Bordeaux-style Red Blend       0.65      0.69      0.67      1719
Bordeaux-style White Blend       0.56      0.16      0.25       249
            Cabernet Franc       0.35      0.14      0.20       320
        Cabernet Sauvignon       0.62      0.67      0.64      2251
           Champagne Blend       0.72      0.31      0.43       335
                Chardonnay       0.59      0.92      0.72      2813
                     Gamay       0.62      0.49      0.54       246
            Gewürztraminer       0.74      0.40      0.52       255
          Grüner Veltliner       0.74      0.63      0.68       309
                    Malbec       0.58      0.49      0.53       657
                    Merlot       0.69      0.28      0.40       748
                  Nebbiolo       0.69      0.82      0.75       696
              Pinot Grigio       0.72      0.35      0.47       251
                Pin

## Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=None))

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy 0.5922251794499557
                            precision    recall  f1-score   support

  Bordeaux-style Red Blend       0.57      0.60      0.59      1719
Bordeaux-style White Blend       0.47      0.42      0.44       249
            Cabernet Franc       0.35      0.34      0.35       320
        Cabernet Sauvignon       0.55      0.57      0.56      2251
           Champagne Blend       0.55      0.47      0.50       335
                Chardonnay       0.70      0.74      0.72      2813
                     Gamay       0.62      0.54      0.58       246
            Gewürztraminer       0.64      0.52      0.57       255
          Grüner Veltliner       0.64      0.58      0.61       309
                    Malbec       0.47      0.45      0.46       657
                    Merlot       0.39      0.38      0.38       748
                  Nebbiolo       0.72      0.71      0.72       696
              Pinot Grigio       0.53      0.52      0.53       251
                Pin

### With Respect to this datset "SVM" model predicts with better Accuracy
Hence considering the same model

## Using the SVM Model on the Validation set

In [9]:
val_set['variety_pred'] = sgd.predict(val_set['review_description'])


In [10]:
val_set.head(10)

user_name    country                                       review_title  \
0   @paulgwine          US  Boedecker Cellars 2011 Athena Pinot Noir (Will...   
1   @wineschach  Argentina  Mendoza Vineyards 2012 Gran Reserva by Richard...   
2       @vboone         US                Prime 2013 Chardonnay (Coombsville)   
3   @wineschach  Argentina  Bodega Cuarto Dominio 2012 Chento Vineyard Sel...   
4  @kerinokeefe      Italy           SassodiSole 2012  Brunello di Montalcino   
5           NaN         US  Mount Pleasant Winery NV Villagio Off Dry Whit...   
6    @vossroger     France  Château Trolliet Lafite 2010 Red (Côtes de Ber...   
7       @vboone         US  Dry Creek Vineyard 2013 Cabernet Sauvignon (Dr...   
8   @wineschach      Chile  Viña Casas Patronales 2010 Mixtura Red (Maule ...   
9  @kerinokeefe      Italy  Contadi Castaldi NV Rosé Sparkling (Franciacorta)   

                                  review_description  \
0  nicely differentiated companion stewart bottli...   
1  charred smoky herbal aromas blackberry trend d...   
2  slightly sour funky earth restrained white age...   
3  concentrated midnightblack malbec delivers aro...   
4  earthy aromas suggesting grilled porcini leath...   
5  timid nose squeaks notes chinese five spice wa...   
6  structured wine berry fruits black currants fi...   
7  remarkable value overdelivering terms varietal...   
8  staunch herbal aromas tobacco olive rubber van...   
9  blend 65 chardonnay 35 pinot nero coppercolore...   

                      designation  points  price          province  \
0                          Athena      88   35.0            Oregon   
1  Gran Reserva by Richard Bonvin      90   60.0  Mendoza Province   
2                             NaN      87   38.0        California   
3       Chento Vineyard Selection      91   20.0  Mendoza Province   
4                             NaN      90   49.0           Tuscany   
5                Villagio Off Dry      82    9.0          Missouri   
6                             NaN      89    NaN  Southwest France   
7                             NaN      88   26.0        California   
8                         Mixtura      88   25.0      Maule Valley   
9                            Rosé      88   24.0          Lombardy   

                 region_1           region_2                   winery  \
0       Willamette Valley  Willamette Valley        Boedecker Cellars   
1                 Mendoza                NaN        Mendoza Vineyards   
2             Coombsville               Napa                    Prime   
3                 Mendoza                NaN    Bodega Cuarto Dominio   
4  Brunello di Montalcino                NaN              SassodiSole   
5                Missouri                NaN    Mount Pleasant Winery   
6       Côtes de Bergerac                NaN  Château Trolliet Lafite   
7        Dry Creek Valley             Sonoma       Dry Creek Vineyard   
8                     NaN                NaN    Viña Casas Patronales   
9            Franciacorta                NaN         Contadi Castaldi   

               variety_pred  
0                Pinot Noir  
1                    Malbec  
2                Chardonnay  
3                    Malbec  
4                Sangiovese  
5                Pinot Gris  
6  Bordeaux-style Red Blend  
7        Cabernet Sauvignon  
8                 Red Blend  
9           Sparkling Blend

In [11]:
val_set.to_excel("Outputfile.xlsx")